THOMAS MCDONNELL QUANT TRADING STRATEGIES HOMEWORK 3

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

Worked with Sean Lin

In [199]:
FR = pd.read_csv('/Users/tmac/Downloads/ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2 2.csv')

In [200]:
FR['per_end_date'] = pd.to_datetime(FR['per_end_date'])
start_date = datetime(2014, 6, 1)
FR = FR[FR['per_end_date'] >= start_date]
end_date = datetime(2022, 1, 31)
FR = FR[FR['per_end_date'] <= end_date]
FR = FR.set_index('ticker')

#Drop all of the tickers that don't have quarterly data
FR = FR[FR['per_type'] == 'Q']

In [201]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')


In [202]:
FC = FC[['ticker','zacks_sector_code', 'per_end_date', 'filing_date']]

In [203]:
FC = FC.drop_duplicates(subset = 'ticker')

#Drop all tickers in the finance, industry and auto sector

FC = FC[~FC['zacks_sector_code'].isin([5.0, 13.0])]
FC = FC.set_index('ticker')

In [204]:
common_tickers = set(FC.index).intersection(set(FR.index))
FR = FR.loc[common_tickers]

In [205]:
FR['tot_debt_tot_equity'] = pd.to_numeric(FR['tot_debt_tot_equity'])

In [206]:
grouped = FR.groupby('ticker')

# Count the number of rows where tot_debt_tot_equity is greater than 0.1
counts = grouped['tot_debt_tot_equity'].apply(lambda x: (x > 0.1).sum())

# Filter out the tickers that don't have more than 83% of their rows with tot_debt_tot_equity greater than 0.1
FR = grouped.filter(lambda x: (x['tot_debt_tot_equity'] > 0.1).sum() > len(x)/1.2)


In [207]:
FR = FR.reset_index()

In [208]:
FR

,ticker,m_ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,invty_turn,rcv_turn,day_sale_rcv,ret_equity,ret_tang_equity,ret_asset,ret_invst,free_cash_flow_per_share,book_val_per_share,oper_cash_flow_per_share
0,BLUD,BLUD,NaN,IMMUCOR,NaN,USD,2014-08-31,Q,NaN,2015,...,0.7533,1.4272,63.0600,-1.1926,0.5124,-0.3042,-0.3663,NaN,NaN,NaN
1,BLUD,BLUD,NaN,IMMUCOR,NaN,USD,2014-11-30,Q,NaN,2015,...,0.7547,1.3695,65.7185,-2.0538,0.8353,-0.5047,-0.6145,NaN,NaN,NaN
2,BLUD,BLUD,NaN,IMMUCOR,NaN,USD,2015-02-28,Q,NaN,2015,...,0.7219,1.3191,68.2270,-10.1374,3.5271,-2.2279,-2.7711,NaN,NaN,NaN
3,BLUD,BLUD,NaN,IMMUCOR,NaN,USD,2015-05-31,Q,NaN,2015,...,0.8863,1.4034,64.1278,-1.7538,0.6068,-0.3911,-0.4847,NaN,NaN,NaN
4,BLUD,BLUD,NaN,IMMUCOR,NaN,USD,2015-08-31,Q,NaN,2016,...,0.7150,1.4724,61.1245,-1.9121,0.6550,-0.4143,-0.5119,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64430,CENTA,CEN3,Central Garden & Pet,Central Garden & Pet Company,NASDAQ,USD,2020-12-31,Q,NaN,2021,...,0.7424,1.8346,49.0562,0.5190,0.8478,0.2307,0.3008,-0.9285,19.7212,-0.6604
64431,CENTA,CEN3,Central Garden & Pet,Central Garden & Pet Company,NASDAQ,USD,2021-03-31,Q,NaN,2021,...,0.9851,1.4694,61.2476,6.3361,9.9004,2.5851,3.4385,-2.7889,20.8951,-2.1788
64432,CENTA,CEN3,Central Garden & Pet,Central Garden & Pet Company,NASDAQ,USD,2021-06-30,Q,NaN,2021,...,1.1438,2.0975,42.9081,6.1801,9.2819,2.4573,3.1644,2.1843,22.3075,3.2093
64433,CENTA,CEN3,Central Garden & Pet,Central Garden & Pet Company,NASDAQ,USD,2021-09-30,Q,NaN,2021,...,0.7681,1.9179,46.9264,-0.2633,-0.4477,-0.1034,-0.1338,3.0863,22.1532,4.5403


In [209]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker', 'per_end_date', 'filing_date']]

In [210]:
FC['per_end_date'] = pd.to_datetime(FC['per_end_date'])
FC['filing_date'] = pd.to_datetime(FC['filing_date'])
merged_df = pd.merge(FR, FC, on=['per_end_date', 'ticker'])
merged_df = merged_df.drop_duplicates()

In [211]:
merged_df = merged_df.set_index('ticker')

In [212]:
merged_df = merged_df.dropna(subset=['filing_date'])

In [213]:
unique_tickers = merged_df.index.unique()
unique_tickers

Index(['BLUD', 'NPTN', 'SALM', 'SJM', 'TFX', 'COL', 'SRT', 'HITI', 'MDP',
       'SIEGY',
       ...
       'NR', 'BOJA', 'MU', 'SHLM', 'RDEN', 'ENSV', 'MSFT', 'UNS', 'GSL',
       'CENTA'],
      dtype='object', name='ticker', length=2331)

In [214]:
merged_df = merged_df.dropna(subset=['filing_date'])

In [360]:
merged_df = merged_df.set_index('ticker')

In [215]:
ticker_counts = {}
for ticker in tqdm(unique_tickers):
    # Get the number of rows for the current ticker
    num_rows = merged_df.loc[merged_df.index == ticker].shape[0]
    # Add the ticker and number of rows to the dictionary
    ticker_counts[ticker] = num_rows


100%|██████████| 2331/2331 [00:05<00:00, 412.88it/s]


In [217]:
#drop all tickers where they dont have at least 28 instances of data
tickers_to_drop = [ticker for ticker in ticker_counts if ticker_counts[ticker] != 31]

# Drop rows where the 'ticker' column is in the list of tickers to drop
merged_df = merged_df[~merged_df.index.isin(tickers_to_drop)]
FR
unique_tickers = merged_df.index.unique().tolist()

In [218]:
len(unique_tickers)

1050

In [226]:
eod_price = pd.DataFrame(columns = unique_tickers)
eod_price['AAPL'] = quandl.get_table('QUOTEMEDIA/PRICES', ticker = ['AAPL'], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2015-01-01', 'lte': '2022-02-01' }, 
                     paginate=True).set_index('date')[['adj_close']]
for ticker in tqdm(unique_tickers):
    df =  quandl.get_table('QUOTEMEDIA/PRICES', ticker = [ticker], api_key = 'uoxc6V3s61czWuHoGLcs',
                    qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                    date = { 'gte': '2015-01-01', 'lte': '2022-02-01' }, 
                     paginate=True)
    eod_price[ticker] = df.set_index('date')[['adj_close']]
eod_price


100%|██████████| 1050/1050 [07:47<00:00,  2.25it/s]


,NPTN,SALM,SJM,TFX,SRT,GME,MWA,HES,PTSI,COP,...,NP,FSTR,CNX,SMG,BYD,XRAY,NR,ENSV,MSFT,CENTA
date,,,,,,,,,,,,,,,,,,,,,
2022-02-01,15.35,3.340000,134.575330,307.915916,4.86,28.150000,12.628230,92.295356,35.8200,87.290001,...,45.537017,15.640000,15.350000,141.981133,60.345321,52.363586,3.78,0.652000,305.858937,43.86
2022-01-31,15.36,3.220000,136.596389,308.602384,5.09,27.232500,12.598816,91.100945,34.9150,84.505570,...,45.048422,15.120000,14.830000,146.346358,58.677560,52.669229,3.55,0.672700,308.058078,43.33
2022-01-28,15.25,2.910000,136.538089,298.315306,5.07,24.477500,12.392921,89.827566,33.4600,85.077714,...,44.696634,15.060000,15.000000,141.835948,58.105192,52.688948,3.57,0.650000,305.363635,42.66
2022-01-27,15.24,2.860000,135.051445,294.047264,5.01,23.380000,12.030154,88.524572,32.1800,85.497285,...,44.383933,15.000000,14.640000,142.223108,56.980193,52.018505,3.59,0.586000,297.022748,42.28
2022-01-26,15.14,2.920000,133.487067,297.210988,4.93,25.815000,12.510575,86.609567,33.4700,83.017998,...,44.882300,14.930000,14.500000,144.071795,58.667691,51.772018,3.75,0.700400,293.922157,41.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-08,3.42,5.569707,81.716899,106.753348,9.63,6.267477,9.047138,61.993145,12.8025,50.172351,...,46.878020,48.577715,31.227027,50.904841,12.380753,50.329624,9.17,23.549988,41.675835,10.08
2015-01-07,3.31,5.469738,80.551696,106.200617,9.49,6.194924,8.596988,60.186676,13.2025,48.951462,...,46.001719,47.899063,30.829863,49.629211,12.062556,49.116975,8.78,23.099988,40.484846,9.88
2015-01-06,3.35,5.326925,78.791838,105.371520,9.60,6.267477,8.588162,60.325635,12.9575,48.626922,...,46.411206,48.066267,30.978800,48.738678,11.725074,47.876125,8.77,22.199989,39.976925,9.67


In [227]:
eod_price = eod_price.dropna(axis=1, how='any')

In [228]:
seed = 42

# Randomly select 300 columns
columns_to_keep = eod_price.sample(n=300, axis=1, random_state=seed)

# Create a new DataFrame with only the selected columns
final_price_data = eod_price[columns_to_keep.columns]

In [229]:
final_price_data

,ABG,HBIO,DLA,PFE,JNPR,INTU,XRAY,PTEN,HBP,ATI,...,CBT,MMM,GTLS,UFPI,MTD,PLPC,NPTN,TAST,WM,SEAS
date,,,,,,,,,,,,,,,,,,,,,
2022-02-01,168.18,5.98,30.96,51.795176,33.143922,556.949605,52.363586,10.454192,9.62,18.810000,...,60.959584,160.556123,125.13,79.835898,1496.43,58.458723,15.35,2.580000,147.401818,61.120000
2022-01-31,160.97,5.82,30.20,51.424304,33.903389,551.388352,52.669229,9.841564,9.12,18.290000,...,53.902034,159.204011,121.87,78.926650,1472.68,59.931310,15.36,2.470000,148.071111,59.580000
2022-01-28,156.90,5.74,29.83,53.024909,32.744714,531.119569,52.688948,9.960137,8.76,18.410000,...,54.519570,156.403892,115.61,78.817936,1430.53,59.377854,15.25,2.360000,145.571107,58.260000
2022-01-27,154.94,5.59,29.20,52.087969,30.622101,513.641344,52.018505,9.782278,8.47,18.630000,...,53.176675,163.174042,110.59,77.582545,1396.45,57.460527,15.24,2.330000,143.947088,57.190000
2022-01-26,158.57,5.62,30.00,51.351744,30.602628,503.968735,51.772018,10.019424,8.71,19.150000,...,53.617772,162.349349,114.76,78.758637,1405.53,59.199958,15.14,2.440000,145.630162,57.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-08,75.50,5.02,9.61,23.029269,19.104615,83.891081,50.329624,13.286603,3.35,30.952223,...,36.346316,128.014967,30.96,16.119636,300.72,44.877724,3.42,7.235685,44.057149,15.867580
2015-01-07,75.01,5.02,9.72,22.568684,18.565406,82.780126,49.116975,13.250571,3.54,30.779894,...,35.253380,125.018589,30.99,15.758333,296.49,44.850936,3.31,6.799910,43.556307,15.968124
2015-01-06,72.69,5.04,9.64,22.263989,18.158925,81.979508,47.876125,13.736996,3.29,30.243759,...,35.146551,124.118893,30.53,15.594667,291.12,43.663364,3.35,6.536629,43.522352,15.858440


In [230]:
final_price_data.to_csv('final_eod_price_data.csv')

In [24]:
SHRS = pd.read_csv('/Users/tmac/Downloads/ZACKS_SHRS_2_99db6fa97ac677f3c0d45a9fa9a70196.csv')

In [25]:
SHRS = SHRS.set_index('ticker')

In [26]:
SHRS['shares_out'] = SHRS[['shares_out']]*1000000


In [433]:
tickers = final_price_data.columns.to_list()

In [232]:
merged_df = merged_df.reset_index()

In [281]:
filtered_df = merged_df[merged_df['ticker'].isin(tickers)]

In [282]:
filtered_df = filtered_df.set_index('ticker')

In [283]:
filtered_df

,index,m_ticker,comp_name,comp_name_2,exchange,currency_code,per_end_date,per_type,per_code,per_fisc_year,...,rcv_turn,day_sale_rcv,ret_equity,ret_tang_equity,ret_asset,ret_invst,free_cash_flow_per_share,book_val_per_share,oper_cash_flow_per_share,filing_date
ticker,,,,,,,,,,,,,,,,,,,,,
NPTN,0,NPTN,NeoPhotonics,NeoPhotonics Corporation,NaN,USD,2014-06-30,Q,NaN,2014,...,0.9773,92.0882,-4.1755,-4.5377,-2.3185,-3.7700,-0.8246,5.0496,-0.6388,2014-08-08
NPTN,1,NPTN,NeoPhotonics,NeoPhotonics Corporation,NaN,USD,2014-09-30,Q,NaN,2014,...,1.0303,87.3500,-1.2171,-1.3128,-0.6781,-1.1094,-0.6344,4.8935,-0.3529,2014-11-10
NPTN,2,NPTN,NeoPhotonics,NeoPhotonics Corporation,NaN,USD,2014-12-31,Q,NaN,2014,...,1.0178,88.4218,0.9940,1.0624,0.5536,0.8789,-0.3572,4.8686,-0.0140,2015-03-16
NPTN,3,NPTN,NeoPhotonics,NeoPhotonics Corporation,NaN,USD,2015-03-31,Q,NaN,2015,...,0.9485,94.8868,0.0615,0.0677,0.0315,0.0527,0.1631,4.9438,0.2254,2015-05-11
NPTN,4,NPTN,NeoPhotonics,NeoPhotonics Corporation,NaN,USD,2015-06-30,Q,NaN,2015,...,1.0754,83.6927,0.8423,0.8997,0.5222,0.8012,0.2570,5.2964,0.4055,2015-08-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSV,32483,ASPN,ENSERVCO,ENSERVCO Corporation,NYSE MKT (ex. AMEX),USD,2020-12-31,Q,NaN,2020,...,1.3803,65.2049,-139.0606,-255.2441,-11.7715,-15.2313,-1.0721,0.4055,-1.0644,2021-03-23
ENSV,32484,ASPN,ENSERVCO,ENSERVCO Corporation,NYSE MKT (ex. AMEX),USD,2021-03-31,Q,NaN,2021,...,1.6214,55.5085,-41.4873,-47.0460,-12.0059,-15.3840,-0.2889,0.8210,-0.2854,2021-05-13
ENSV,32485,ASPN,ENSERVCO,ENSERVCO Corporation,NYSE MKT (ex. AMEX),USD,2021-06-30,Q,NaN,2021,...,1.9600,45.9184,-16.0542,-18.2038,-4.7219,-5.8969,-0.1970,0.7807,-0.1856,2021-08-05


In [252]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
df = pd.DataFrame(index=date_range)

# Add columns from final_price_data
df[final_price_data.columns] = None

df = df.rename_axis('filing_date')

In [270]:
filtered_debt = filtered_df[['tot_debt_tot_equity', 'filing_date']]
filtered_debt

,tot_debt_tot_equity,filing_date
ticker,,
NPTN,0.2958,2014-08-08
NPTN,0.2911,2014-11-10
NPTN,0.2892,2015-03-16
NPTN,0.4069,2015-05-11
NPTN,0.2147,2015-08-10
...,...,...
ENSV,8.8180,2021-03-23
ENSV,1.8537,2021-05-13
ENSV,1.9189,2021-08-05


In [271]:
pivot_table = filtered_debt.pivot_table(values='tot_debt_tot_equity', index='filing_date', columns='ticker')

In [272]:
pivot_table = pivot_table.reindex(columns = df.columns)

In [256]:
pivot_table

,ABG,HBIO,DLA,PFE,JNPR,INTU,XRAY,PTEN,HBP,ATI,...,CBT,MMM,GTLS,UFPI,MTD,PLPC,NPTN,TAST,WM,SEAS
filing_date,,,,,,,,,,,,,,,,,,,,,
2014-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-23,2.1929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [273]:
for i in tqdm(pivot_table.index):
    for ticker in pivot_table.columns:
        df[ticker][i] = pivot_table[ticker][i]

100%|██████████| 1256/1256 [00:44<00:00, 28.39it/s]


In [274]:
df.ffill(inplace=True)

In [275]:
df = df[df.index >= datetime(2015, 1,)]

In [276]:
debt_to_equity = df

In [265]:
debt_to_equity.to_csv('debt_to_equity.csv')

In [278]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
roi = pd.DataFrame(index=date_range)

# Add columns from final_price_data
roi[final_price_data.columns] = None

roi = roi.rename_axis('filing_date')

In [290]:
filtered_roi = filtered_df[['ret_invst', 'filing_date']]
pivot_table_roi = filtered_roi.pivot_table(values='ret_invst', index='filing_date', columns='ticker')
pivot_table_roi = pivot_table.reindex(columns = roi.columns)
for i in tqdm(pivot_table_roi.index):
    for ticker in pivot_table_roi.columns:
        roi[ticker][i] = pivot_table_roi[ticker][i]

100%|██████████| 1259/1259 [00:34<00:00, 36.74it/s]


In [303]:
filtered_roi

,ret_invst,filing_date
ticker,,
NPTN,-3.7700,2014-08-08
NPTN,-1.1094,2014-11-10
NPTN,0.8789,2015-03-16
NPTN,0.0527,2015-05-11
NPTN,0.8012,2015-08-10
...,...,...
ENSV,-15.2313,2021-03-23
ENSV,-15.3840,2021-05-13
ENSV,-5.8969,2021-08-05


In [292]:
roi.ffill(inplace=True)
roi = roi[roi.index >= datetime(2015, 1, 1)]


In [294]:
roi.to_csv('roi.csv')

In [1]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
eps = pd.DataFrame(index=date_range)

# Add columns from final_price_data
eps[final_price_data.columns] = None

eps = eps.rename_axis('filing_date')

NameError: name 'pd' is not defined

In [298]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker','diluted_net_eps', 'per_end_date', 'filing_date']]

In [300]:
FC = FC.set_index('ticker')

In [301]:
FC = FC.loc[tickers]

In [310]:
pivot_table_eps = FC.pivot_table(values='diluted_net_eps', index='filing_date', columns='ticker')
pivot_table_eps = pivot_table_eps.reindex(columns = eps.columns)
for i in tqdm(pivot_table_eps.index):
    for ticker in pivot_table_eps.columns:
        eps[ticker][i] = pivot_table_eps[ticker][i]
eps.ffill(inplace=True)
eps = eps[eps.index >= datetime(2015, 1, 1)]

100%|██████████| 1259/1259 [00:35<00:00, 35.87it/s]


In [312]:
eps.to_csv('eps.csv')

In [473]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
lt_debt = pd.DataFrame(index=date_range)

# Add columns from final_price_data
lt_debt[final_price_data.columns] = None

lt_debt = lt_debt.rename_axis('filing_date')

In [474]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker','net_lterm_debt', 'per_end_date', 'filing_date']]

In [475]:
FC = FC.set_index('ticker')

In [476]:
pivot_table_lt_debt = FC.pivot_table(values='net_lterm_debt', index='filing_date', columns='ticker')
pivot_table_lt_debt = pivot_table_lt_debt.reindex(columns = eps.columns)
for i in tqdm(pivot_table_lt_debt.index):
    for ticker in pivot_table_lt_debt.columns:
        lt_debt[ticker][i] = pivot_table_lt_debt[ticker][i]
lt_debt.ffill(inplace=True)
lt_debt = lt_debt[lt_debt.index >= datetime(2015, 1, 1)]

100%|██████████| 2519/2519 [03:14<00:00, 12.97it/s]


In [477]:
lt_debt

,ABG,HBIO,DLA,PFE,JNPR,INTU,XRAY,PTEN,HBP,ATI,...,CBT,MMM,GTLS,UFPI,MTD,PLPC,NPTN,TAST,WM,SEAS
filing_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,NaN,-1.55,-4.761,3705.0,346.1,NaN,-84.9,-7.500,12.9,-414.7,...,-10.0,-487.0,-2.813,NaN,74.448,19.011,-0.546,-0.756,-28.0,-42.025
2015-01-02,NaN,-1.55,-4.761,3705.0,346.1,NaN,-84.9,-7.500,12.9,-414.7,...,-10.0,-487.0,-2.813,NaN,74.448,19.011,-0.546,-0.756,-28.0,-42.025
2015-01-03,NaN,-1.55,-4.761,3705.0,346.1,NaN,-84.9,-7.500,12.9,-414.7,...,-10.0,-487.0,-2.813,NaN,74.448,19.011,-0.546,-0.756,-28.0,-42.025
2015-01-04,NaN,-1.55,-4.761,3705.0,346.1,NaN,-84.9,-7.500,12.9,-414.7,...,-10.0,-487.0,-2.813,NaN,74.448,19.011,-0.546,-0.756,-28.0,-42.025
2015-01-05,NaN,-1.55,-4.761,3705.0,346.1,NaN,-84.9,-7.500,12.9,-414.7,...,-10.0,-487.0,-2.813,NaN,74.448,19.011,-0.546,-0.756,-28.0,-42.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-28,7.3,-3.00,-16.253,-4.0,-423.8,-338.0,-282.0,-62.525,-17.1,664.7,...,-22.0,-449.0,-344.100,150.0,389.144,-12.655,11.283,-20.717,-809.0,-107.506
2022-01-29,7.3,-3.00,-16.253,-4.0,-423.8,-338.0,-282.0,-62.525,-17.1,664.7,...,-22.0,-449.0,-344.100,150.0,389.144,-12.655,11.283,-20.717,-809.0,-107.506
2022-01-30,7.3,-3.00,-16.253,-4.0,-423.8,-338.0,-282.0,-62.525,-17.1,664.7,...,-22.0,-449.0,-344.100,150.0,389.144,-12.655,11.283,-20.717,-809.0,-107.506


In [478]:
lt_debt.to_csv('lt_debt.csv')

In [327]:
SHRS = SHRS[['per_end_date', 'shares_out']]

In [337]:
SHRS = SHRS.reset_index()

In [420]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker','net_lterm_debt', 'per_end_date', 'filing_date']]

In [421]:
SHRS['per_end_date'] = pd.to_datetime(SHRS['per_end_date'])
FC['per_end_date'] = pd.to_datetime(FC['per_end_date'])
FC = FC.reset_index()

In [458]:
merged = pd.merge(SHRS, FC, on=['per_end_date', 'ticker'])
merged = merged.drop_duplicates()

In [459]:
merged = merged.set_index('ticker')

In [460]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
shares = pd.DataFrame(index=date_range)

# Add columns from final_price_data
shares[final_price_data.columns] = None
shares = shares.rename_axis('filing_date')

In [461]:
merged = merged[['shares_out', 'filing_date']]
merged

,shares_out,filing_date
ticker,,
A,296040000.0,2022-09-01
A,298710000.0,2022-05-31
A,300370000.0,2022-03-03
A,302000000.0,2021-12-17
A,302000000.0,2021-12-17
...,...,...
ZZ,90980000.0,2007-02-23
ZZ,90980000.0,2007-02-23
ZZ,91000000.0,2006-10-11


In [463]:
merged = merged.loc[tickers]

In [464]:
merged = merged.reset_index()

In [465]:
shares

,ABG,HBIO,DLA,PFE,JNPR,INTU,XRAY,PTEN,HBP,ATI,...,CBT,MMM,GTLS,UFPI,MTD,PLPC,NPTN,TAST,WM,SEAS
filing_date,,,,,,,,,,,,,,,,,,,,,
2014-07-17,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2014-07-18,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2014-07-19,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2014-07-20,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2014-07-21,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-28,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2022-01-29,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2022-01-30,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [466]:
pivot_table_shares = merged.pivot_table(values='shares_out', index='filing_date', columns='ticker')
pivot_table_shares = pivot_table_shares.reindex(columns = shares.columns)
for i in tqdm(pivot_table_shares.index):
    for ticker in pivot_table_shares.columns:
        shares[ticker][i] = pivot_table_shares[ticker][i]
shares.ffill(inplace=True)
shares = shares[shares.index >= datetime(2015, 1, 1)]

100%|██████████| 2671/2671 [03:26<00:00, 12.95it/s]


In [467]:
shares

,ABG,HBIO,DLA,PFE,JNPR,INTU,XRAY,PTEN,HBP,ATI,...,CBT,MMM,GTLS,UFPI,MTD,PLPC,NPTN,TAST,WM,SEAS
filing_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,29770000.0,32300000.0,7880000.0,6.340860e+09,451190000.0,285500000.0,141780000.0,146410000.0,24570000.0,108720000.0,...,64580000.0,640820000.0,30480000.0,60210000.0,28890000.0,5390000.0,32550000.0,35220000.0,466400000.0,90010000.0
2015-01-02,29770000.0,32300000.0,7880000.0,6.340860e+09,451190000.0,285500000.0,141780000.0,146410000.0,24570000.0,108720000.0,...,64580000.0,640820000.0,30480000.0,60210000.0,28890000.0,5390000.0,32550000.0,35220000.0,466400000.0,90010000.0
2015-01-03,29770000.0,32300000.0,7880000.0,6.340860e+09,451190000.0,285500000.0,141780000.0,146410000.0,24570000.0,108720000.0,...,64580000.0,640820000.0,30480000.0,60210000.0,28890000.0,5390000.0,32550000.0,35220000.0,466400000.0,90010000.0
2015-01-04,29770000.0,32300000.0,7880000.0,6.340860e+09,451190000.0,285500000.0,141780000.0,146410000.0,24570000.0,108720000.0,...,64580000.0,640820000.0,30480000.0,60210000.0,28890000.0,5390000.0,32550000.0,35220000.0,466400000.0,90010000.0
2015-01-05,29770000.0,32300000.0,7880000.0,6.340860e+09,451190000.0,285500000.0,141780000.0,146410000.0,24570000.0,108720000.0,...,64580000.0,640820000.0,30480000.0,60210000.0,28890000.0,5390000.0,32550000.0,35220000.0,466400000.0,90010000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-28,19340000.0,40760000.0,6970000.0,5.606690e+09,325180000.0,283170000.0,218550000.0,215110000.0,27400000.0,127250000.0,...,56720000.0,578640000.0,36380000.0,61890000.0,22990000.0,4900000.0,52690000.0,51270000.0,418320000.0,79180000.0
2022-01-29,19340000.0,40760000.0,6970000.0,5.606690e+09,325180000.0,283170000.0,218550000.0,215110000.0,27400000.0,127250000.0,...,56720000.0,578640000.0,36380000.0,61890000.0,22990000.0,4900000.0,52690000.0,51270000.0,418320000.0,79180000.0
2022-01-30,19340000.0,40760000.0,6970000.0,5.606690e+09,325180000.0,283170000.0,218550000.0,215110000.0,27400000.0,127250000.0,...,56720000.0,578640000.0,36380000.0,61890000.0,22990000.0,4900000.0,52690000.0,51270000.0,418320000.0,79180000.0


In [479]:
shares.to_csv('shares.csv')

,net_lterm_debt,per_end_date,filing_date
ticker,,,
ABG,NaN,2006-12-31,2007-03-08
ABG,NaN,2007-12-31,2008-02-29
ABG,NaN,2008-12-31,2009-03-16
ABG,NaN,2009-12-31,2010-03-01
ABG,NaN,2010-12-31,2011-02-28
...,...,...,...
SEAS,-107.506,2021-09-30,2021-11-15
SEAS,-110.506,2021-12-31,2022-02-28
SEAS,-3.000,2022-03-31,2022-05-06
